In [1]:
# read neccessary libraries
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

pd.options.display.max_rows = 20

np.set_printoptions(precision = 4, suppress = True)

In [2]:
def get_team_name(soup):
    title = soup.find('title')
    pattern = '<title>(.*) Statistics | Premier League</title>'
    regex = re.compile(pattern, flags=re.IGNORECASE)
    team_name = regex.match(str(title)).groups()[0]
    return team_name

# Return the filtered club name for the given index 
def get_premier_teams(target_teams, i):
    pattern = '<h4 class="clubName">(.*)<'
    regex = re.compile(pattern, flags=re.IGNORECASE)
    result = regex.match(str(target_teams[i])).groups()
    return result[0]

In [3]:
# Get the 20 Premier League teams (first division)

premier_teams = []
url = "https://www.premierleague.com/clubs?se=210"
res = requests.get(url)
soup = BeautifulSoup(res.content)
target_teams = soup.find_all('h4', attrs={'clubName'})

for i in range(0,20):
    premier_teams.append(get_premier_teams(target_teams, i))

premier_teams

['Arsenal',
 'Aston Villa',
 'AFC Bournemouth',
 'Brighton and Hove Albion',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Leicester City',
 'Liverpool',
 'Manchester City',
 'Manchester United',
 'Newcastle United',
 'Norwich City',
 'Sheffield United',
 'Southampton',
 'Tottenham Hotspur',
 'Watford',
 'West Ham United',
 'Wolverhampton Wanderers']

In [52]:
headers = ['Team']
team_stats = []
premier_teams_index = []

# Iterate through clubs using numbers 1-46 in the URL
# Since the 20 Premier League teams are found in mixed order with second division teams, we have to extract all
for team in range(1,20):
    url = "https://www.premierleague.com/clubs/" + str(team) + "/club/stats"
    res = requests.get(url)  
    soup = BeautifulSoup(res.content)

    team_name = get_team_name(soup)
    
    # Filter out FC from the football clubs' names
    if 'FC' in team_name:
        team_name = team_name[:-3]
        
    # Filter out teams that aren't in the Premier League
    if team_name not in premier_teams:
        continue
    
    # Append the indices (for URL) of the Premier League teams (first division)
    premier_teams_index.append(team)    
    
    # Find all the stats that are enclosed in 'div' classed as normalStat
    all_stats = soup.find_all('div', attrs={'normalStat'})                     

    stats = [team_name]
    
    for i in range(13,26): 
        
        # Filter the first child from each 'div' tag which contains the stat name
        span = all_stats[i].findChildren()[0]

        # Only need to extract headers once
        if team == 1:                     
            pattern = '<span class="stat">(.*)<(.*)'
            regex = re.compile(pattern, flags=re.IGNORECASE)
            result = regex.match(str(span)).groups()
            headers.append(result[0].strip())
        
        # Extract the statistical data
        pattern = '(.*)\\n( *)(.*)'
        regex = re.compile(pattern, flags=re.IGNORECASE)
        result = regex.match(str(span)).groups()
        
        # Use float only for 'Goals conceded per match'
        if i != 15:
            data_value = int(result[2].replace(',','').replace('%',''))
        else:
            data_value = float(result[2].replace(',','').replace('%',''))
        stats.append(data_value)
        
    team_stats.append(stats)


In [53]:
pd.DataFrame(team_stats, columns=headers).set_index('Team')

,Clean sheets,Goals conceded,Goals conceded per match,Saves,Tackles,Tackle success %,Blocked shots,Interceptions,Clearances,Headed Clearance,Aerial Battles/Duels Won,Errors leading to goal,Own goals
Team,,,,,,,,,,,,,
Arsenal,405,1047,0.99,793,10307,73,2097,8009,14057,5196,36668,113,41
Aston Villa,272,1231,1.30,425,8515,75,1190,6235,14410,4566,28839,76,46
Chelsea,425,1034,0.97,602,10229,74,2392,6636,13180,4984,35793,66,36
Crystal Palace,103,592,1.44,758,4819,71,788,3823,7162,3834,20539,52,16
Everton,327,1346,1.27,707,9960,74,1871,7071,17276,6078,36260,85,52
Liverpool,403,1060,1.00,547,11194,74,2237,6670,14167,4993,35968,89,39
Manchester City,285,1002,1.15,510,10143,74,2193,7141,14135,4892,33999,62,38
Manchester United,448,956,0.90,692,10033,73,2098,7181,15473,5866,34378,59,39
Norwich City,70,555,1.64,315,3035,72,516,2253,5444,2059,11097,43,18
